# RASA 项目中数据处理

在一个 RASA 项目中，数据必须遵循 [RASA Training data format](https://rasa.com/docs/rasa/nlu/training-data-format/) 才能进行训练，本片文章是一个将 CSV 文件转化成目标格式的例子。

原数据在 [raw/orders.csv](raw/orders.csv)

## 设置 Jupyter Notebook

设置 Jupyter Notebook 的输出，只打印有用的信息，忽略没用的 warning 信息。

如果使用 Jupyter Notebook, 请查看[Using Jupyter Notebook](https://rasa.com/docs/rasa/api/jupyter-notebooks/)


In [ ]:
%matplotlib inline

import nest_asyncio
nest_asyncio.apply()
# Logging settings
import logging, io, json, warnings, pprint
# logging.basicConfig(level="ERROR")
# warnings.filterwarnings('ignore')
pp = pprint.PrettyPrinter(indent=2)

## 准备实验数据 (CSV to JSON)

将一个 CSV 文件转化成一个可以供 RASA 使用的 JSON 训练集.

In [ ]:
import pandas as pd
import json

# 读取文件
raw_df = pd.read_csv('raw/orders.csv', delimiter=',', usecols=['refund_Info', 'category_id'])

预览 10 行数据

In [ ]:
raw_df.head(10)

数据中有些 column 是空，这里涉及到对缺失数据的处理。另外 category 字段为数字，会对分类造成错误，变成 `category_x` 这样的文本类型

In [ ]:
# remove the column which refund_Info column is empty
raw_df = raw_df[raw_df.refund_Info.notnull()]

# rename the column name
raw_df = raw_df.rename({'refund_Info': 'text', 'category_id': 'intent'}, axis='columns')

# add prefix category_ to category_id
raw_df['intent'] = raw_df['intent'].apply(lambda x: 'category_' + str(x))

转化成 RASA JSON 格式

In [ ]:
examples = raw_df.to_dict(orient='records')

output = {
    "rasa_nlu_data": {
        "common_examples": examples
    }
}

输出到文件 `data_exp2/result.json` 

In [ ]:
with open('data_exp2/result.json', 'w', encoding='utf-8') as fp:
    json.dump(output, fp, ensure_ascii=False, indent=2)

## (可选)转化成易读数据集 (JSON to Markdown)

RASA 支持 JSON 和 Markdown 两种格式，都可以用来训练，Markdown 更方便阅读，如果有需要，可以将JSON文件转化成Markdown

In [ ]:
from rasa.nlu.convert import convert_training_data

convert_training_data(data_file="data_exp2/result.json", out_file="data_exp2/result.md", output_format="md", language="zh")

## 验证数据是否符合 RASA 项目的训练要求

> [Valiate Files](https://rasa.com/docs/rasa/user-guide/validate-files/) 文档上给的代码是错误的。需要提交 Pull Request 来修复。

> // TODO, 下面这个验证会失败，但不影响实验.

In [ ]:
import logging
from rasa import utils
from rasa.core.validator import Validator
from rasa.importers.importer import TrainingDataImporter

dataImporter = TrainingDataImporter.load_nlu_importer_from_config(config_path="config.yml",
                                                                  training_data_paths="data_exp2/result.md")

validator = await Validator.from_importer(importer=dataImporter)

validator.verify_all()

## 训练 NLU 模型

根据如下代码训练 NLU 模型，[参考文档](https://github.com/RasaHQ/rasa-workshop-pydata-dc/blob/master/rasa-pydatadc-workshop-completed.ipynb),
该文档为旧版本的 rasa 语法。需要根据新版本的语法，并且改成中文训练

In [ ]:
config = "config.yml"
training_files = "data_exp2/result.json"
domain = "domain.yml"
output = "models/"

import rasa
model_path = rasa.train(config, [training_files], output)
print(model_path)

## 查看 NLU 模型的输出结果

手动查看一些 NLU 训练模型的输出结果



实验结束。Have a nice day!